# Day 3: Gear Ratios

[https://adventofcode.com/2023/day/3](https://adventofcode.com/2023/day/3)

## Description

### Part One

You and the Elf eventually reach a [gondola lift](https://en.wikipedia.org/wiki/Gondola_lift) station; he says the gondola lift will take you up to the _water source_, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can _add up all the part numbers_ in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently _any number adjacent to a symbol_, even diagonally, is a "part number" and should be included in your sum. (Periods (`.`) do not count as a symbol.)

Here is an example engine schematic:

    467..114..
    ...*......
    ..35..633.
    ......#...
    617*......
    .....+.58.
    ..592.....
    ......755.
    ...$.*....
    .664.598..
    

In this schematic, two numbers are _not_ part numbers because they are not adjacent to a symbol: `114` (top right) and `58` (middle right). Every other number is adjacent to a symbol and so _is_ a part number; their sum is _`4361`_.

Of course, the actual engine schematic is much larger. _What is the sum of all of the part numbers in the engine schematic?_

In [278]:
engine_schematic = open("data.txt").read().split()

In [279]:
def parse_schematic(schematic: list):
    number_positions = {}
    symbol_positions = []

    for row, line in enumerate(schematic):
        col = 0
        while col < len(line):
            char = line[col]
            if char.isdigit():
                # Start of a number
                start = col
                while col < len(line) and line[col].isdigit():
                    col += 1
                end = col - 1
                number = line[start:col]
                # Check if the number is already in the dict and append the position if it is
                if number in number_positions:
                    number_positions[number].append((row, start, end))
                else:
                    number_positions[number] = [(row, start, end)]
            elif char == ".":
                # Skip '.'
                col += 1
            else:
                # Position of a symbol
                symbol_positions.append((row, col))
                col += 1

    return number_positions, symbol_positions

In [280]:
# Parsing the schematic
number_positions_dict, symbol_positions_list = parse_schematic(engine_schematic)

In [283]:
def find_adjacent_numbers_to_symbols(number_positions, symbol_positions, rows, cols):
    adjacent_numbers = []

    # Function to check if a number is adjacent to a symbol
    def is_adjacent(number_pos, symbol_pos):
        num_row, num_start_col, num_end_col = number_pos
        sym_row, sym_col = symbol_pos

        # Check if symbol is in a row adjacent (or same) to the number
        if sym_row in range(max(0, num_row - 1), min(rows, num_row + 2)):
            # Check if symbol is in a column adjacent (or same) to the number
            if sym_col in range(max(0, num_start_col - 1), min(cols, num_end_col + 2)):
                return True
        return False

    # Check each number against each symbol position
    for number, num_pos_list in number_positions.items():
        for num_pos in num_pos_list:
            for sym_pos in symbol_positions:
                if is_adjacent(num_pos, sym_pos):
                    adjacent_numbers.append(number)
                    break  # No need to check other positions

    return adjacent_numbers

In [284]:
rows = len(engine_schematic)  # number of rows
cols = len(engine_schematic[0])  # number of cols

# Finding numbers adjacent to symbols
adjacent_numbers = find_adjacent_numbers_to_symbols(
    number_positions_dict, symbol_positions_list, rows, cols
)

# Answer
print("Answer #1:", sum(map(int, adjacent_numbers)))

Answer #1: 507214


### Part Two

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A _gear_ is any `*` symbol that is adjacent to _exactly two part numbers_. Its _gear ratio_ is the result of <span title="They're magic gears.">multiplying</span> those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

    467..114..
    ...*......
    ..35..633.
    ......#...
    617*......
    .....+.58.
    ..592.....
    ......755.
    ...$.*....
    .664.598..
    

In this schematic, there are _two_ gears. The first is in the top left; it has part numbers `467` and `35`, so its gear ratio is `16345`. The second gear is in the lower right; its gear ratio is `451490`. (The `*` adjacent to `617` is _not_ a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces _`467835`_.

_What is the sum of all of the gear ratios in your engine schematic?_


#### Solution

In [340]:
import string

# Get all punctuation symbols and remove '.'
symbols_without_dot = string.punctuation.replace(".", "")

def parse_schematic(schematic: list):
    number_positions = {}
    symbol_positions = {}

    for row, line in enumerate(schematic):
        col = 0
        while col < len(line):
            char = line[col]
            if char.isdigit():
                # Start of a number
                start = col
                while col < len(line) and line[col].isdigit():
                    col += 1
                end = col - 1
                number = line[start:col]
                # Check if the number is already in the dict and append the position if it is
                number_positions[number] = number_positions.get(number, []) + [
                    (row, start, end)
                ]
            elif char in symbols_without_dot:
                # Position of a symbol
                symbol_positions[char] = symbol_positions.get(char, []) + [(row, col)]
                col += 1
            else:
                # Skip '.'
                col += 1

    return number_positions, symbol_positions

In [341]:
# Parsing the schematic
number_positions_dict, symbol_positions_dict = parse_schematic(engine_schematic)

In [342]:
def find_gears(number_positions, symbol_positions):
    valid_numbers = []
    # Function to check if if two numbers are adjacent to a '*'
    def is_adjacent(number_pos, symbol_pos):
        num_row, num_start_col, num_end_col = number_pos
        sym_row, sym_col = symbol_pos

        # Check if symbol is in a row adjacent (or same) to the number
        if sym_row in range(max(0, num_row - 1), min(rows, num_row + 2)):
            # Check if symbol is in a column adjacent (or same) to the number
            if sym_col in range(max(0, num_start_col - 1), min(cols, num_end_col + 2)):
                return True
        return False

    # Check each number against each symbol position
    for sym_pos in symbol_positions:
        adjacent_numbers = []
        for number, num_pos_list in number_positions.items():                
            for num_pos in num_pos_list:
                if is_adjacent(num_pos, sym_pos):
                    adjacent_numbers.append(int(number))
        if len(adjacent_numbers) == 2:
            valid_numbers.append(adjacent_numbers[0] * adjacent_numbers[1])

    return valid_numbers

In [343]:
# Find which symbol
symbol_positions_list = symbol_positions_dict["*"]

print("Answer #2:",sum(find_gears(number_positions_dict, symbol_positions_list)))

Answer #2: 72553319
